# Intermediate SQL with SQLite in Python

In this lesson, we will work with the **Sakila** database—a sample database originally created by MySQL and ported to SQLite. The Sakila database contains a rich schema (including tables such as `actor`, `film`, `film_actor`, etc.) that provides real-world data to explore advanced SQL techniques.

### What You'll Learn:
- How to connect to a SQLite database file (`sakila.db`) using Python's `sqlite3` module.
- How to explore the Sakila database schema by listing tables and previewing data.
- How to write SQL queries using JOINs, aggregate functions, GROUP BY, and HAVING.
- How to apply these techniques with partner practice exercises.

**Note:** Ensure the `sakila.db` file is in your working directory before running the notebook.

In [2]:
import sqlite3
import pandas as pd

# Set the path to the SQLite database
db_path = "sakila.db"

# Connect to SQLite database
conn = sqlite3.connect(db_path)
print("Connected to SQLite Sakila database!")

Connected to SQLite Sakila database!


### Exploring the Database Schema

Before diving into queries, it's useful to know which tables are available in the Sakila database. We'll list all tables by querying the SQLite master table.

In [4]:
# List all tables in the database
tables = pd.read_sql("""SELECT name 
                        FROM sqlite_master 
                        WHERE type='table';""", conn)

print("Tables in the database:")
print(tables)

Tables in the database:
               name
0             actor
1           country
2              city
3           address
4          language
5          category
6          customer
7              film
8        film_actor
9     film_category
10        film_text
11        inventory
12            staff
13            store
14          payment
15           rental
16        customers
17  sqlite_sequence
18       test_table


### Viewing Sample Data from the 'actor' Table

Let's preview a few rows from the `actor` table to see what kind of data it holds.

Select the following columns:
- actor_id
- first_name
- last_name

Connect two string columns with || ' ' || and create an alias with "AS"

In [14]:
# Define SQL query to fetch the first 5 rows from the actor table.
# second command joins the elements together and renames the column
query_actor = """SELECT actor_id, first_name ||' '|| last_name AS actor_name 
                 FROM actor
                 LIMIT 5; 
              """
# limits actor_id, not index 
# Execute the query using pd.read_sql.
df_actor = pd.read_sql(query_actor, conn)

print("Sample rows from the 'actor' table:")
print(df_actor)

Sample rows from the 'actor' table:
   actor_id           actor_name
0         1     PENELOPE GUINESS
1         2        NICK WAHLBERG
2         3             ED CHASE
3         4       JENNIFER DAVIS
4         5  JOHNNY LOLLOBRIGIDA


### Viewing Sample Data from the 'film' Table

Let's preview a few rows from the `film` table to see what kind of data it holds.

Select the following columns:
- film_id
- title
- length
- rating

In [25]:
# Define SQL query to fetch the first 5 rows from the actor table.
query_film = """SELECT film_id, title, length, rating
                FROM film
                LIMIT 5;
             """

# Execute the query using pd.read_sql.
df_film = pd.read_sql(query_film, conn)

print("Sample rows from the 'film' table:")
print(df_film)

Sample rows from the 'film' table:
   film_id             title  length rating
0        1  ACADEMY DINOSAUR      86     PG
1        2    ACE GOLDFINGER      48      G
2        3  ADAPTATION HOLES      50  NC-17
3        4  AFFAIR PREJUDICE     117      G
4        5       AFRICAN EGG     130      G


### Viewing Sample Data from the 'film_actor' Table

Let's preview a few rows from the `film_actor` table to see what kind of data it holds.

Select all columns.

In [13]:
# Define SQL query to fetch the first 5 rows from the actor table.
query_film_actor = """SELECT * 
                      FROM film_actor
                      LIMIT 5;
                   """

# Execute the query using pd.read_sql.
df_film_actor = pd.read_sql(query_film_actor, conn)

print("Sample rows from the 'film_actor' table:")
print(df_film_actor)

Sample rows from the 'film_actor' table:
   actor_id  film_id          last_update
0         1        1  2021-03-06 15:52:45
1         1       23  2021-03-06 15:52:45
2         1       25  2021-03-06 15:52:45
3         1      106  2021-03-06 15:52:45
4         1      140  2021-03-06 15:52:45


### JOIN Query: Actors and Their Films

Next, we'll join three tables—`actor`, `film_actor`, and `film`—to list film titles along with the full names of actors who appear in them.

- **Tables Involved:**
  - `actor`: Contains actor details.
  - `film_actor`: A bridge table linking actors to films.
  - `film`: Contains film details.

The join connects:
- `actor.actor_id` with `film_actor.actor_id`
- `film_actor.film_id` with `film.film_id`

Let's see the first 10 actor-film associations.

In [ ]:
# Define SQL query for actor-film associations.
# join actor table with film table via film_actor
join_query = """SELECT actor.first_name, actor.last_name, film.title
                FROM actor 
                JOIN film_actor ON actor.actor_id = film_actor.actor_id
                JOIN film ON film.film_id = film_actor.film_id
                ORDER BY actor.actor_id
                LIMIT 10;
             """
# DESC to go from bottom of list of actor ids instead of ASC for top
# if create an alias using AS need to change code throughout the rest where it is referenced
# defaults order ascending primary key of first table 

# Execute the query using pd.read_sql.
df_join = pd.read_sql(join_query, conn)

print("Actor - Film Associations (first 10 rows):")
print(df_join)

Actor - Film Associations (first 10 rows):
  first_name last_name                  title
0   PENELOPE   GUINESS       ACADEMY DINOSAUR
1   PENELOPE   GUINESS   ANACONDA CONFESSIONS
2   PENELOPE   GUINESS            ANGELS LIFE
3   PENELOPE   GUINESS  BULWORTH COMMANDMENTS
4   PENELOPE   GUINESS          CHEAPER CLYDE
5   PENELOPE   GUINESS     COLOR PHILADELPHIA
6   PENELOPE   GUINESS        ELEPHANT TROJAN
7   PENELOPE   GUINESS    GLEAMING JAWBREAKER
8   PENELOPE   GUINESS         HUMAN GRAFFITI
9   PENELOPE   GUINESS         KING EVOLUTION


### Aggregation Query: Counting Actors per Film

Now, we'll count the number of actors featured in each film. This query uses:
- **JOIN:** To connect `film` and `film_actor`.
- **GROUP BY:** To group data by film title.
- **HAVING:** To filter films with more than 3 actors.

This helps identify films with larger casts.

In [23]:
# Define SQL query for counting actors per film.
agg_query = """SELECT f.title, COUNT(film_actor.actor_id) AS actor_count
               FROM film as f
               LEFT JOIN film_actor ON f.film_id = film_actor.film_id
               GROUP BY f.film_id
               HAVING actor_count > 3
               ORDER BY actor_count DESC
               LIMIT 10;
            """

# Execute the query using pd.read_sql.
df_agg = pd.read_sql(agg_query, conn)

print("Films with more than 3 actors (top 10):")
print(df_agg)

Films with more than 3 actors (top 10):
                 title  actor_count
0     LAMBS CINCINATTI           15
1    BOONDOCK BALLROOM           13
2          CHITTY LOCK           13
3           CRAZY HOME           13
4      DRACULA CRYSTAL           13
5      MUMMY CREATURES           13
6            RANDOM GO           13
7         ARABIA DOGMA           12
8  HELLFIGHTERS SIERRA           12
9     LESSON CLEOPATRA           12


# Group Exercises
- Find a neighbor, yes, move around and make a new friend!
- Work through the following practice exercises together.

# Partner Practice Exercise

Now that we've seen several examples of JOINs and aggregation queries, it's time for you to practice with a partner. Complete the following tasks using the Sakila database:

1. **Task 1: Write a JOIN Query**
   - Write a query to retrieve a list of films along with the full names of the actors (first and last name) who acted in them.
   - *Hint:* Join the `film`, `film_actor`, and `actor` tables. Use string concatenation (e.g., `first_name || ' ' || last_name`) for full names.

2. **Task 2: Write an Aggregation Query**
   - Write a query to find the average film length for each film category.
   - *Hint:* Join the `film`, `film_category`, and `category` tables, then use the `AVG()` function with `GROUP BY`.

3. **Task 3: Challenge Task (Optional)**
   - Combine your results: Write a query to list film titles, the count of actors in each film, and the film category. Filter to show only films with more than 5 actors.
   - *Hint:* You might need to use multiple joins and group by both film title and category.

Discuss your approaches with your partner, compare your results, and be ready to review a few solutions as a class.

In [ ]:
# Task 1: Retrieve a list of films along with the full names of the actors.
task1_query = """

              """

# Execute the query using pd.read_sql.
df_task1 = pd.read_sql(task1_query, conn)

print("Task 1: Films with Actor Full Names (first 20 rows):")
print(df_task1)

In [27]:
# Task 2: Find the average film length for each film category.
task2_query = """SELECT c.name AS category_name, AVG(f.length) AS avg_length
                 FROM film as f
                 JOIN film_category AS fc ON f.film_id = fc.film_id
                 JOIN category AS c ON fc.category_id - c.category_id
                 GROUP BY c.name
                 ORDER BY avg_length DESC
              """

# Execute the query using pd.read_sql.
df_task2 = pd.read_sql(task2_query, conn)

print("Task 2: Average Film Length per Category:")
print(df_task2)

Task 2: Average Film Length per Category:
   category_name  avg_length
0    Documentary  115.747854
1         Sci-Fi  115.731629
2       Children  115.621277
3      Animation  115.572805
4            New  115.550694
5         Action  115.522436
6       Classics  115.489926
7         Horror  115.437500
8         Travel  115.390244
9          Music  115.359326
10        Family  115.308271
11        Comedy  115.237792
12         Drama  114.904051
13       Foreign  114.765912
14         Games  114.455804
15        Sports  114.238661


In [ ]:
# Task 3: List film titles, count of actors, and film category for films with more than 5 actors.
task3_query = """

            """

# Execute the query using pd.read_sql.
df_task3 = pd.read_sql(task3_query, conn)

print("Task 3: Films with more than 5 Actors (showing film title, actor count, category):")
print(df_task3)

## Close the connection to the database.
Always close the connection when you're done to free up resources.

In [24]:
# Close the connection
#conn.close()